In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 5
latent_dim = 256
num_samples = 10000

data_path="deu.txt"

In [2]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [4]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for input', max_encoder_seq_length)
print('Max sequence length of output', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 83
Max sequence length for input 15
Max sequence length of output 51


In [5]:
input_token_index = dict(
    [(char, i) for i ,char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i ,char in enumerate(target_characters)])

In [6]:
encoder_input_data = np.zeros(
            (len(input_texts),max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros(
            (len(input_texts),max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros(
            (len(input_texts),max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [7]:
for i, (input_text, target_text) in enumerate (zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]]=1.
    encoder_input_data[i, t + 1:, input_token_index[' ']]=1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] =1.
        if t > 0:
            decoder_target_data[i,t - 1,target_token_index[char]]=1.
    
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']]=1.

In [8]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c =encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [9]:



encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/5
125/125 [==============================] - 52s 286ms/step - loss: 0.5407 - accuracy: 0.8442 - val_loss: 0.6657 - val_accuracy: 0.8081
Epoch 2/5
125/125 [==============================] - 41s 329ms/step - loss: 0.5059 - accuracy: 0.8538 - val_loss: 0.6381 - val_accuracy: 0.8175
Epoch 3/5
125/125 [==============================] - 31s 252ms/step - loss: 0.4790 - accuracy: 0.8616 - val_loss: 0.6159 - val_accuracy: 0.8249
Epoch 4/5
125/125 [==============================] - 33s 268ms/step - loss: 0.4560 - accuracy: 0.8679 - val_loss: 0.5972 - val_accuracy: 0.8288
Epoch 5/5
 37/125 [=======>......................] - ETA: 21s - loss: 0.4394 - accuracy: 0.8726